In [1]:
import pandas as pd
import numpy as np
import os
import re
os.chdir("C:/Users/tommy/Documents/final_sem/Applied Data Science/Project2/data/tables")
from pyspark.sql import SparkSession
from pyspark.shell import spark
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
import matplotlib.pyplot as plt

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.0
      /_/

Using Python version 3.9.12 (main, Apr  4 2022 05:22:27)
Spark context Web UI available at http://192.168.1.6:4040
Spark context available as 'sc' (master = local[*], app id = local-1661865912056).
SparkSession available as 'spark'.


# consumer dataset

In [2]:
# read csv file
consumer = spark.read.csv('consumer.csv', header = True)
consumer

name,address,state,postcode,gender,consumer_id
Yolanda Williams,413 Haney Gardens...,WA,6935,Female,1195503
Mary Smith,3764 Amber Oval,NSW,2782,Female,179208
Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
Lindsay Jimenez,00653 Davenport C...,NSW,2780,Female,154128
Rebecca Blanchard,9271 Michael Mano...,WA,6355,Female,712975
Karen Chapman,2706 Stewart Oval...,NSW,2033,Female,407340
Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
Stephen Williams,6804 Wright Crest...,WA,6056,Male,448088
Stephanie Reyes,5813 Denise Land ...,NSW,2482,Female,650435
Jillian Gonzales,461 Ryan Common S...,VIC,3220,Female,1058499


# user detail dataset

In [3]:
user_detail = spark.read.parquet("consumer_user_details.parquet")
user_detail

user_id,consumer_id
1,1195503
2,179208
3,1194530
4,154128
5,712975
6,407340
7,511685
8,448088
9,650435
10,1058499


# transaction dataset

In [4]:
path = "C:/Users/tommy/Documents/final_sem/Applied Data Science/Project2/data/tables/transactions_20210228_20210827_snapshot/"
list_files = os.listdir(path)
list_files = list_files[1:(len(list_files)-1)]
list_files

['order_datetime=2021-02-28',
 'order_datetime=2021-03-01',
 'order_datetime=2021-03-02',
 'order_datetime=2021-03-03',
 'order_datetime=2021-03-04',
 'order_datetime=2021-03-05',
 'order_datetime=2021-03-06',
 'order_datetime=2021-03-07',
 'order_datetime=2021-03-08',
 'order_datetime=2021-03-09',
 'order_datetime=2021-03-10',
 'order_datetime=2021-03-11',
 'order_datetime=2021-03-12',
 'order_datetime=2021-03-13',
 'order_datetime=2021-03-14',
 'order_datetime=2021-03-15',
 'order_datetime=2021-03-16',
 'order_datetime=2021-03-17',
 'order_datetime=2021-03-18',
 'order_datetime=2021-03-19',
 'order_datetime=2021-03-20',
 'order_datetime=2021-03-21',
 'order_datetime=2021-03-22',
 'order_datetime=2021-03-23',
 'order_datetime=2021-03-24',
 'order_datetime=2021-03-25',
 'order_datetime=2021-03-26',
 'order_datetime=2021-03-27',
 'order_datetime=2021-03-28',
 'order_datetime=2021-03-29',
 'order_datetime=2021-03-30',
 'order_datetime=2021-03-31',
 'order_datetime=2021-04-01',
 'order_da

In [6]:
# import modules
from pyspark.sql import SparkSession
import functools
 
# explicit function
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

# insert datetime
file_name = os.listdir(path+ list_files[0])[1]
transaction = spark.read.parquet(path+ list_files[0] +"/" + file_name)
for i in list_files[1:]:
    file_name = os.listdir(path + i)[1]
    tmp = spark.read.parquet(path+ i +"/" + file_name)
    transaction = unionAll([transaction, tmp] )

In [8]:
transaction

user_id,merchant_abn,dollar_value,order_id
1,28000487688,133.22689421562643,0c37b3f7-c7f1-48c...
18485,62191208634,79.13140006851712,9e18b913-0465-4fd...
1,83690644458,30.441348317517228,40a2ff69-ea34-465...
18488,39649557865,962.8133405407585,f4c1a5ae-5b76-40d...
2,80779820715,48.12397733548124,cd09bdd6-f56d-489...
18489,43186523025,98.14878546968934,9008a98e-1b02-4de...
3,29566626791,46.33087226118639,26b7574e-81c2-455...
18490,93558142492,232.83335268750145,2bda0665-796f-4f2...
3,32361057556,87.34942171371054,633a7656-2fcc-4b8...
18491,64974914166,130.12601873970038,4bc15338-83eb-43d...


# merchant dataset

In [9]:
# preprocessing tags(NLP:singular/plural,upper/lower... + One-hot econding, cosine-similarity -> grouping -> naming type of merchants)
path1 = 'C:/Users/tommy/Documents/final_sem/Applied Data Science/Project2/data/tables/'
merchant = spark.read.parquet(path1 + "tbl_merchants.parquet")
merchant

name,tags,merchant_abn
Felis Limited,"((furniture, home...",10023283211
Arcu Ac Orci Corp...,"([cable, satellit...",10142254217
Nunc Sed Company,"([jewelry, watch,...",10165489824
Ultricies Digniss...,"([wAtch, clock, a...",10187291046
Enim Condimentum PC,([music shops - m...,10192359162
Fusce Company,"[(gift, card, nov...",10206519221
Aliquam Enim Inco...,"[(computers, comP...",10255988167
Ipsum Primis Ltd,"[[watch, clock, a...",10264435225
Pede Ultrices Ind...,([computer progra...,10279061213
Nunc Inc.,"[(furniture, home...",10323485998


In [10]:
merchant.select("tags")

tags
"((furniture, home..."
"([cable, satellit..."
"([jewelry, watch,..."
"([wAtch, clock, a..."
([music shops - m...
"[(gift, card, nov..."
"[(computers, comP..."
"[[watch, clock, a..."
([computer progra...
"[(furniture, home..."


# To-Do
### Externel Dataset
    what r we going to use
    * Census Dataset (based on postcode)
    * Lat,Lon data (geospatial related dataset)
    * interest rate + research paper
### Merchant Data
    * preprocessing tags  
        split data format: list+list, list+tuple, tuple+tuple, tuple+list
        NLP:singular/plural,upper/lower... + One-hot econding, cosine-similarity -> grouping -> naming type of merchants

### Consumer Dataset
    * find Lat, Lon using postcode
    * drop name, address

### Transaction Dataset
    * insert datetime
    * insert dayofweeks(Mon,Tue...)
    * 

### Merge
    * Final step! Preprocess each dataset -> merge